# AI Health & Fitness Agent

* An AI-powered health and fitness agent that provides personalized dietary and exercise recommendations based on user profiles.
* The agent creates customized meal plans, workout routines, and wellness advice considering age, weight, height, activity level, and fitness goals.
* Features include dietary preference support (vegetarian, keto, gluten-free), fitness goal targeting (weight loss, muscle gain, endurance), and comprehensive health planning with nutritional guidance.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/ai_health_fitness_agent.ipynb)


# Dependencies

In [ ]:
!pip install praisonai streamlit google-generativeai pandas numpy matplotlib seaborn

# Setup Key

In [2]:
import os

# Set your Gemini API key
gemini_key = "Enter your api key here"  # Get from https://aistudio.google.com/apikey

# Set environment variable
os.environ["GOOGLE_API_KEY"] = gemini_key

# Model selection
model_choice = "gemini-2.0-flash-exp"  # Options: "gemini-2.0-flash-exp", "gemini-1.5-pro", "gemini-1.5-flash"

print("✅ API key configured!")
print(f"✅ Using model: {model_choice}")

✅ API key configured!
✅ Using model: gemini-2.0-flash-exp


# Tools

In [3]:
import math
from typing import Dict, Any, List

class BMICalculatorTool:
    def __init__(self):
        self.bmi_categories = {
            "Underweight": (0, 18.5),
            "Normal weight": (18.5, 25),
            "Overweight": (25, 30),
            "Obese": (30, float('inf'))
        }

    def calculate_bmi(self, weight_kg: float, height_cm: float) -> Dict[str, Any]:
        """Calculate BMI and provide health insights"""
        try:
            # Convert height from cm to meters
            height_m = height_cm / 100

            # Calculate BMI
            bmi = weight_kg / (height_m ** 2)

            # Determine category
            category = "Unknown"
            for cat, (min_val, max_val) in self.bmi_categories.items():
                if min_val <= bmi < max_val:
                    category = cat
                    break

            # Calculate ideal weight range
            min_ideal_weight = 18.5 * (height_m ** 2)
            max_ideal_weight = 25 * (height_m ** 2)

            # Health recommendations based on BMI
            recommendations = self._get_bmi_recommendations(bmi, category)

            return {
                "success": True,
                "bmi": round(bmi, 2),
                "category": category,
                "ideal_weight_range": {
                    "min": round(min_ideal_weight, 1),
                    "max": round(max_ideal_weight, 1)
                },
                "recommendations": recommendations,
                "health_risks": self._get_health_risks(category)
            }
        except Exception as e:
            return {"error": f"Error calculating BMI: {str(e)}"}

    def _get_bmi_recommendations(self, bmi: float, category: str) -> List[str]:
        """Get health recommendations based on BMI"""
        recommendations = []

        if category == "Underweight":
            recommendations = [
                "Increase caloric intake with nutrient-dense foods",
                "Include protein-rich foods in every meal",
                "Consider strength training to build muscle mass",
                "Eat frequent, smaller meals throughout the day"
            ]
        elif category == "Normal weight":
            recommendations = [
                "Maintain current healthy eating habits",
                "Continue regular physical activity",
                "Focus on balanced nutrition",
                "Monitor weight regularly"
            ]
        elif category == "Overweight":
            recommendations = [
                "Create a moderate caloric deficit",
                "Increase physical activity",
                "Focus on whole foods and vegetables",
                "Consider working with a nutritionist"
            ]
        elif category == "Obese":
            recommendations = [
                "Consult with healthcare professionals",
                "Start with low-impact exercises",
                "Focus on sustainable lifestyle changes",
                "Consider medical weight loss programs"
            ]

        return recommendations

    def _get_health_risks(self, category: str) -> List[str]:
        """Get health risks associated with BMI category"""
        risks = {
            "Underweight": [
                "Weakened immune system",
                "Nutritional deficiencies",
                "Osteoporosis risk",
                "Reduced muscle mass"
            ],
            "Normal weight": [
                "Lower risk of chronic diseases",
                "Better overall health outcomes"
            ],
            "Overweight": [
                "Increased risk of type 2 diabetes",
                "Higher blood pressure",
                "Heart disease risk",
                "Joint problems"
            ],
            "Obese": [
                "High risk of cardiovascular disease",
                "Type 2 diabetes",
                "Sleep apnea",
                "Joint and mobility issues"
            ]
        }
        return risks.get(category, [])

# Custom Calorie Calculator Tool
class CalorieCalculatorTool:
    def __init__(self):
        self.activity_multipliers = {
            "Sedentary": 1.2,
            "Lightly Active": 1.375,
            "Moderately Active": 1.55,
            "Very Active": 1.725,
            "Extremely Active": 1.9
        }

    def calculate_calories(self, age: int, weight_kg: float, height_cm: float,
                          sex: str, activity_level: str, goal: str) -> Dict[str, Any]:
        """Calculate daily calorie needs based on goals"""
        try:
            # Calculate BMR using Mifflin-St Jeor Equation
            if sex.lower() == "male":
                bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
            else:
                bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161

            # Calculate TDEE (Total Daily Energy Expenditure)
            activity_multiplier = self.activity_multipliers.get(activity_level, 1.2)
            tdee = bmr * activity_multiplier

            # Adjust calories based on goal
            goal_adjustments = {
                "Lose Weight": -500,  # 500 calorie deficit
                "Gain Muscle": 300,   # 300 calorie surplus
                "Maintain": 0,        # No change
                "Endurance": 200,     # Small surplus for energy
                "Strength Training": 250  # Moderate surplus
            }

            calorie_adjustment = goal_adjustments.get(goal, 0)
            target_calories = tdee + calorie_adjustment

            # Calculate macronutrient breakdown
            macros = self._calculate_macros(target_calories, goal)

            return {
                "success": True,
                "bmr": round(bmr),
                "tdee": round(tdee),
                "target_calories": round(target_calories),
                "calorie_adjustment": calorie_adjustment,
                "macros": macros,
                "meal_distribution": self._get_meal_distribution(target_calories)
            }
        except Exception as e:
            return {"error": f"Error calculating calories: {str(e)}"}

    def _calculate_macros(self, calories: float, goal: str) -> Dict[str, Any]:
        """Calculate macronutrient breakdown"""
        if goal == "Lose Weight":
            protein_pct = 0.35  # Higher protein for satiety
            fat_pct = 0.30
            carb_pct = 0.35
        elif goal == "Gain Muscle":
            protein_pct = 0.30
            fat_pct = 0.25
            carb_pct = 0.45  # Higher carbs for energy
        else:  # Maintain or other goals
            protein_pct = 0.25
            fat_pct = 0.30
            carb_pct = 0.45

        protein_cals = calories * protein_pct
        fat_cals = calories * fat_pct
        carb_cals = calories * carb_pct

        return {
            "protein": {
                "calories": round(protein_cals),
                "grams": round(protein_cals / 4),
                "percentage": protein_pct * 100
            },
            "fat": {
                "calories": round(fat_cals),
                "grams": round(fat_cals / 9),
                "percentage": fat_pct * 100
            },
            "carbs": {
                "calories": round(carb_cals),
                "grams": round(carb_cals / 4),
                "percentage": carb_pct * 100
            }
        }

    def _get_meal_distribution(self, total_calories: float) -> Dict[str, int]:
        """Get calorie distribution across meals"""
        return {
            "breakfast": round(total_calories * 0.25),
            "lunch": round(total_calories * 0.30),
            "dinner": round(total_calories * 0.30),
            "snacks": round(total_calories * 0.15)
        }

# Custom Exercise Recommendation Tool
class ExerciseRecommendationTool:
    def __init__(self):
        self.exercise_database = {
            "strength": {
                "beginner": ["Push-ups", "Squats", "Planks", "Lunges", "Wall sits"],
                "intermediate": ["Dumbbell rows", "Deadlifts", "Bench press", "Pull-ups", "Dips"],
                "advanced": ["Barbell squats", "Overhead press", "Power cleans", "Muscle-ups", "Handstand push-ups"]
            },
            "cardio": {
                "beginner": ["Walking", "Cycling", "Swimming", "Elliptical", "Light jogging"],
                "intermediate": ["Running", "HIIT", "Rowing", "Jump rope", "Stair climbing"],
                "advanced": ["Sprint intervals", "Mountain biking", "CrossFit", "Boxing", "Rock climbing"]
            },
            "flexibility": {
                "beginner": ["Static stretching", "Yoga basics", "Foam rolling", "Cat-cow stretches", "Child's pose"],
                "intermediate": ["Dynamic stretching", "Vinyasa yoga", "Pilates", "Hip openers", "Shoulder mobility"],
                "advanced": ["Advanced yoga poses", "Contortion", "Gymnastics", "Dance", "Martial arts"]
            }
        }

    def get_exercise_plan(self, goal: str, fitness_level: str,
                         available_time: int, equipment: List[str]) -> Dict[str, Any]:
        """Generate personalized exercise plan"""
        try:
            # Determine exercise focus based on goal
            if goal == "Lose Weight":
                focus = ["cardio", "strength"]
                cardio_ratio = 0.6
            elif goal == "Gain Muscle":
                focus = ["strength", "cardio"]
                cardio_ratio = 0.3
            elif goal == "Endurance":
                focus = ["cardio", "strength"]
                cardio_ratio = 0.7
            elif goal == "Strength Training":
                focus = ["strength", "flexibility"]
                cardio_ratio = 0.2
            else:  # Stay Fit
                focus = ["strength", "cardio", "flexibility"]
                cardio_ratio = 0.4

            # Generate workout plan
            workout_plan = {
                "warm_up": self._get_warmup_routine(fitness_level),
                "main_workout": self._get_main_workout(focus, fitness_level, available_time, cardio_ratio),
                "cool_down": self._get_cooldown_routine(fitness_level),
                "frequency": self._get_workout_frequency(goal, fitness_level),
                "progression": self._get_progression_plan(fitness_level, goal)
            }

            return {
                "success": True,
                "goal": goal,
                "fitness_level": fitness_level,
                "workout_plan": workout_plan,
                "tips": self._get_exercise_tips(goal, fitness_level)
            }
        except Exception as e:
            return {"error": f"Error generating exercise plan: {str(e)}"}

    def _get_warmup_routine(self, fitness_level: str) -> List[str]:
        """Get appropriate warmup routine"""
        warmups = {
            "beginner": [
                "5-10 minutes light walking or cycling",
                "Arm circles (10 forward, 10 backward)",
                "Leg swings (10 each leg)",
                "Gentle stretching"
            ],
            "intermediate": [
                "5-10 minutes dynamic warmup",
                "Jumping jacks (30 seconds)",
                "High knees (30 seconds)",
                "Butt kicks (30 seconds)",
                "Arm and leg mobility exercises"
            ],
            "advanced": [
                "10-15 minutes dynamic warmup",
                "Jump rope (2-3 minutes)",
                "Burpees (5-10 reps)",
                "Mountain climbers (30 seconds)",
                "Full body mobility routine"
            ]
        }
        return warmups.get(fitness_level, warmups["beginner"])

    def _get_main_workout(self, focus: List[str], fitness_level: str,
                         available_time: int, cardio_ratio: float) -> Dict[str, Any]:
        """Generate main workout based on focus and time"""
        workout = {}

        # Calculate time allocation
        cardio_time = int(available_time * cardio_ratio)
        strength_time = available_time - cardio_time

        if "cardio" in focus:
            workout["cardio"] = {
                "duration": cardio_time,
                "exercises": self.exercise_database["cardio"][fitness_level][:3],
                "intensity": "Moderate to High" if fitness_level != "beginner" else "Low to Moderate"
            }

        if "strength" in focus:
            workout["strength"] = {
                "duration": strength_time,
                "exercises": self.exercise_database["strength"][fitness_level][:4],
                "sets": 3 if fitness_level != "beginner" else 2,
                "reps": "8-12" if fitness_level != "beginner" else "10-15"
            }

        if "flexibility" in focus:
            workout["flexibility"] = {
                "duration": 10,
                "exercises": self.exercise_database["flexibility"][fitness_level][:3]
            }

        return workout

    def _get_cooldown_routine(self, fitness_level: str) -> List[str]:
        """Get cooldown routine"""
        cooldowns = {
            "beginner": [
                "5 minutes light walking",
                "Gentle stretching",
                "Deep breathing exercises"
            ],
            "intermediate": [
                "5-10 minutes light activity",
                "Static stretching",
                "Foam rolling"
            ],
            "advanced": [
                "10 minutes light activity",
                "Comprehensive stretching",
                "Recovery techniques"
            ]
        }
        return cooldowns.get(fitness_level, cooldowns["beginner"])

    def _get_workout_frequency(self, goal: str, fitness_level: str) -> Dict[str, Any]:
        """Get recommended workout frequency"""
        if goal == "Lose Weight":
            frequency = 4 if fitness_level != "beginner" else 3
        elif goal == "Gain Muscle":
            frequency = 4 if fitness_level != "beginner" else 3
        elif goal == "Endurance":
            frequency = 5 if fitness_level != "beginner" else 4
        else:
            frequency = 3

        return {
            "workouts_per_week": frequency,
            "rest_days": 7 - frequency,
            "recommendation": f"Work out {frequency} times per week with {7-frequency} rest days"
        }

    def _get_progression_plan(self, fitness_level: str, goal: str) -> List[str]:
        """Get progression recommendations"""
        if fitness_level == "beginner":
            return [
                "Start with 2-3 workouts per week",
                "Focus on proper form and technique",
                "Gradually increase intensity over 4-6 weeks",
                "Consider working with a trainer initially"
            ]
        elif fitness_level == "intermediate":
            return [
                "Increase workout frequency to 4-5 times per week",
                "Add more challenging exercises",
                "Incorporate progressive overload",
                "Consider periodization training"
            ]
        else:  # advanced
            return [
                "Maintain 5-6 workouts per week",
                "Focus on specific skill development",
                "Incorporate advanced training techniques",
                "Consider competition or performance goals"
            ]

    def _get_exercise_tips(self, goal: str, fitness_level: str) -> List[str]:
        """Get exercise-specific tips"""
        tips = [
            "Always warm up before exercising",
            "Listen to your body and rest when needed",
            "Stay hydrated throughout your workout",
            "Focus on proper form over weight or speed"
        ]

        if goal == "Lose Weight":
            tips.extend([
                "Combine cardio and strength training",
                "Keep workouts consistent",
                "Monitor your heart rate during cardio"
            ])
        elif goal == "Gain Muscle":
            tips.extend([
                "Progressive overload is key",
                "Allow adequate rest between sets",
                "Focus on compound movements"
            ])

        return tips

# YAML Prompt

In [4]:
# YAML Prompt
yaml_prompt = """
name: "AI Health & Fitness Agent"
description: "Expert health and fitness advisor with personalized planning capabilities"
instructions:
  - "You are an expert health and fitness advisor that creates personalized plans based on user profiles"
  - "Consider user's age, weight, height, activity level, and fitness goals when making recommendations"
  - "Provide safe, evidence-based advice for diet and exercise"
  - "Create detailed meal plans with nutritional information and calorie counts"
  - "Design workout routines appropriate for the user's fitness level and goals"
  - "Always prioritize safety and recommend consulting healthcare professionals when appropriate"
  - "Provide clear explanations for why specific recommendations are made"
  - "Consider dietary restrictions and preferences in meal planning"
  - "Include progression plans and tips for long-term success"

tools:
  - name: "BMICalculatorTool"
    description: "Calculates BMI, provides health insights, and generates weight management recommendations"
  - name: "CalorieCalculatorTool"
    description: "Calculates daily calorie needs, macronutrient breakdown, and meal distribution based on goals"
  - name: "ExerciseRecommendationTool"
    description: "Generates personalized exercise plans with warm-up, main workout, and cool-down routines"

output_format:
  - "Provide comprehensive health assessments with BMI and calorie calculations"
  - "Include detailed meal plans with nutritional breakdown"
  - "Offer personalized exercise routines with proper progression"
  - "Include safety considerations and professional consultation advice"
  - "Provide actionable tips and long-term success strategies"
  - "Use clear, structured formatting with sections for diet and fitness"

temperature: 0.3
max_tokens: 4000
model: "gemini-2.0-flash-exp"
"""

print("✅ YAML Prompt configured!")

✅ YAML Prompt configured!


# Main

In [5]:
# Main Application (Google Colab Version)
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Any, List
import os

# Initialize tools
bmi_tool = BMICalculatorTool()
calorie_tool = CalorieCalculatorTool()
exercise_tool = ExerciseRecommendationTool()

print("🏋️‍♂️ AI Health & Fitness Agent")
print("Get personalized health and fitness plans tailored to your goals!")

# User profile input
print("\n👤 Your Health Profile")
print("Please provide your information:")

# Get user input
age = int(input("Age: "))
weight = float(input("Weight (kg): "))
height = float(input("Height (cm): "))
sex = input("Sex (Male/Female/Other): ")
activity_level = input("Activity Level (Sedentary/Lightly Active/Moderately Active/Very Active/Extremely Active): ")
fitness_goals = input("Fitness Goals (Lose Weight/Gain Muscle/Endurance/Stay Fit/Strength Training): ")
dietary_preferences = input("Dietary Preferences (Vegetarian/Keto/Gluten Free/Low Carb/Dairy Free/None): ")

print(f"\n✅ Profile created for {age}-year-old {sex} with {fitness_goals} goals")

# Health Assessment
print("\n📊 Health Assessment")
print("Calculating BMI and health metrics...")

bmi_result = bmi_tool.calculate_bmi(weight, height)
if "error" not in bmi_result:
    print(f"✅ BMI: {bmi_result['bmi']} ({bmi_result['category']})")
    print(f"�� Ideal weight range: {bmi_result['ideal_weight_range']['min']} - {bmi_result['ideal_weight_range']['max']} kg")

    print("\n💡 Health Recommendations:")
    for rec in bmi_result['recommendations']:
        print(f"  - {rec}")

    print("\n⚠️ Health Risks:")
    for risk in bmi_result['health_risks']:
        print(f"  - {risk}")
else:
    print(f"❌ Error calculating BMI: {bmi_result['error']}")

# Calorie Calculation
print("\n🔥 Calorie Analysis")
print("Calculating daily calorie needs...")

calorie_result = calorie_tool.calculate_calories(
    age, weight, height, sex, activity_level, fitness_goals
)

if "error" not in calorie_result:
    print(f"✅ BMR (Basal Metabolic Rate): {calorie_result['bmr']} calories/day")
    print(f"✅ TDEE (Total Daily Energy Expenditure): {calorie_result['tdee']} calories/day")
    print(f"�� Target Calories: {calorie_result['target_calories']} calories/day")

    print("\n🍽️ Macronutrient Breakdown:")
    macros = calorie_result['macros']
    print(f"  Protein: {macros['protein']['grams']}g ({macros['protein']['percentage']}%)")
    print(f"  Fat: {macros['fat']['grams']}g ({macros['fat']['percentage']}%)")
    print(f"  Carbs: {macros['carbs']['grams']}g ({macros['carbs']['percentage']}%)")

    print("\n⏰ Meal Distribution:")
    meals = calorie_result['meal_distribution']
    print(f"  Breakfast: {meals['breakfast']} calories")
    print(f"  Lunch: {meals['lunch']} calories")
    print(f"  Dinner: {meals['dinner']} calories")
    print(f"  Snacks: {meals['snacks']} calories")
else:
    print(f"❌ Error calculating calories: {calorie_result['error']}")

# Exercise Plan
print("\n💪 Exercise Plan")
print("Generating personalized workout routine...")

# Determine fitness level based on activity level
fitness_level_map = {
    "Sedentary": "beginner",
    "Lightly Active": "beginner",
    "Moderately Active": "intermediate",
    "Very Active": "intermediate",
    "Extremely Active": "advanced"
}
fitness_level = fitness_level_map.get(activity_level, "intermediate")

exercise_result = exercise_tool.get_exercise_plan(
    fitness_goals, fitness_level, 60, ["bodyweight", "dumbbells"]
)

if "error" not in exercise_result:
    print(f"✅ Fitness Level: {fitness_level}")
    print(f"🎯 Goal: {fitness_goals}")

    workout_plan = exercise_result['workout_plan']

    print("\n🔥 Warm-up Routine:")
    for warmup in workout_plan['warm_up']:
        print(f"  - {warmup}")

    print("\n🏋️‍♂️ Main Workout:")
    main_workout = workout_plan['main_workout']

    if 'cardio' in main_workout:
        cardio = main_workout['cardio']
        print(f"  Cardio ({cardio['duration']} minutes):")
        for exercise in cardio['exercises']:
            print(f"    - {exercise}")

    if 'strength' in main_workout:
        strength = main_workout['strength']
        print(f"  Strength Training ({strength['duration']} minutes):")
        for exercise in strength['exercises']:
            print(f"    - {exercise} ({strength['sets']} sets, {strength['reps']} reps)")

    if 'flexibility' in main_workout:
        flexibility = main_workout['flexibility']
        print(f"  Flexibility ({flexibility['duration']} minutes):")
        for exercise in flexibility['exercises']:
            print(f"    - {exercise}")

    print("\n🧘‍♀️ Cool-down Routine:")
    for cooldown in workout_plan['cool_down']:
        print(f"  - {cooldown}")

    print(f"\n�� Workout Frequency: {workout_plan['frequency']['recommendation']}")

    print("\n💡 Exercise Tips:")
    for tip in exercise_result['tips']:
        print(f"  - {tip}")

    print("\n📈 Progression Plan:")
    for progression in workout_plan['progression']:
        print(f"  - {progression}")
else:
    print(f"❌ Error generating exercise plan: {exercise_result['error']}")

# Personalized Recommendations
print("\n🎯 Personalized Recommendations")

# Dietary recommendations based on goals and preferences
print("\n🍽️ Dietary Recommendations:")
if fitness_goals == "Lose Weight":
    print("  - Create a moderate caloric deficit")
    print("  - Focus on high-protein foods for satiety")
    print("  - Include plenty of vegetables and fiber")
    print("  - Stay hydrated with water")
elif fitness_goals == "Gain Muscle":
    print("  - Ensure adequate protein intake (1.6-2.2g per kg body weight)")
    print("  - Include complex carbohydrates for energy")
    print("  - Don't forget healthy fats")
    print("  - Consider protein timing around workouts")
elif fitness_goals == "Endurance":
    print("  - Focus on carbohydrates for energy")
    print("  - Include moderate protein for recovery")
    print("  - Stay well-hydrated")
    print("  - Consider sports nutrition for long sessions")

# Dietary preference adjustments
if dietary_preferences != "None":
    print(f"\n🥗 {dietary_preferences} Adjustments:")
    if dietary_preferences == "Vegetarian":
        print("  - Include plant-based protein sources (beans, lentils, tofu)")
        print("  - Consider B12 supplementation")
        print("  - Ensure adequate iron from plant sources")
    elif dietary_preferences == "Keto":
        print("  - Keep carbs under 50g per day")
        print("  - Focus on healthy fats and proteins")
        print("  - Monitor ketone levels")
    elif dietary_preferences == "Gluten Free":
        print("  - Choose gluten-free grains (quinoa, rice, oats)")
        print("  - Read labels carefully")
        print("  - Ensure adequate fiber intake")

# Safety and Professional Advice
print("\n⚠️ Important Safety Considerations:")
print("  - Consult with a healthcare professional before starting any new diet or exercise program")
print("  - Start slowly and gradually increase intensity")
print("  - Listen to your body and rest when needed")
print("  - Stay hydrated and maintain proper nutrition")
print("  - Consider working with certified trainers or nutritionists")

# Progress Tracking
print("\n📊 Progress Tracking Recommendations:")
print("  - Track your weight weekly (same time, same conditions)")
print("  - Take progress photos monthly")
print("  - Keep a food and exercise journal")
print("  - Monitor energy levels and mood")
print("  - Set realistic, measurable goals")

# Sample Meal Plan
print("\n🍽️ Sample Meal Plan (1 Day):")
print("Based on your calorie target and preferences")

if "target_calories" in calorie_result:
    target_cals = calorie_result['target_calories']
    meals = calorie_result['meal_distribution']

    print(f"\n🌅 Breakfast ({meals['breakfast']} calories):")
    if dietary_preferences == "Vegetarian":
        print("  - Greek yogurt with berries and nuts")
        print("  - Whole grain toast with avocado")
        print("  - Green tea or coffee")
    else:
        print("  - Scrambled eggs with spinach")
        print("  - Whole grain toast")
        print("  - Fresh fruit")

    print(f"\n🌞 Lunch ({meals['lunch']} calories):")
    if dietary_preferences == "Keto":
        print("  - Grilled chicken salad with olive oil")
        print("  - Mixed greens and vegetables")
        print("  - Nuts and seeds")
    else:
        print("  - Quinoa bowl with vegetables")
        print("  - Lean protein (chicken/fish/tofu)")
        print("  - Healthy dressing")

    print(f"\n🌙 Dinner ({meals['dinner']} calories):")
    print("  - Lean protein with vegetables")
    print("  - Complex carbohydrates")
    print("  - Healthy fats")

    print(f"\n�� Snacks ({meals['snacks']} calories):")
    print("  - Fresh fruit or vegetables")
    print("  - Nuts or seeds")
    print("  - Greek yogurt or cottage cheese")

# Footer
print("\n" + "="*50)
print("🏋️‍♂️ Powered by AI Health & Fitness Agent | Built with PraisonAI")
print("💡 Remember: Consistency is key to achieving your health and fitness goals!")

🏋️‍♂️ AI Health & Fitness Agent
Get personalized health and fitness plans tailored to your goals!

👤 Your Health Profile
Please provide your information:
Age: 25
Weight (kg): 43
Height (cm): 152
Sex (Male/Female/Other): Female
Activity Level (Sedentary/Lightly Active/Moderately Active/Very Active/Extremely Active): Lightly Active
Fitness Goals (Lose Weight/Gain Muscle/Endurance/Stay Fit/Strength Training): Gain Muscle
Dietary Preferences (Vegetarian/Keto/Gluten Free/Low Carb/Dairy Free/None): None

✅ Profile created for 25-year-old Female with Gain Muscle goals

📊 Health Assessment
Calculating BMI and health metrics...
✅ BMI: 18.61 (Normal weight)
�� Ideal weight range: 42.7 - 57.8 kg

💡 Health Recommendations:
  - Maintain current healthy eating habits
  - Continue regular physical activity
  - Focus on balanced nutrition
  - Monitor weight regularly

⚠️ Health Risks:
  - Lower risk of chronic diseases
  - Better overall health outcomes

🔥 Calorie Analysis
Calculating daily calorie ne